In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward


In [2]:
def initialize_parameters_deep(layer_dims):
    
    parameters = {}
    L = len(layer_dims)
    
    for l in range(1, L):
        parameters["W"+str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        parameters["b"+str(l)] = np.zeros((layer_dims[l], 1))
        
    return parameters

In [3]:
#Forward Propagation For A Single Layer

def linear_forward(A_prev, W, b):
    
    Z = np.dot(W, A_prev) + b
    cache  = (A_prev, W, B)
    
    return Z, cache

In [4]:
#Combining linear forward and its activation in a function for a layer

def linear_activation(A_prev, W, b, activation):
    
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        
    cache = (linear_cache, activation_cache)
    
    return A, cache

In [5]:
#Forward Propagation For All L-Layers

def l_model_forward(X, parameters):
    
    caches = []
    A = X
    L = len(parameters)/2
    
    for l in range(1,L):
        
        A_prev = A
        A, cache = linear_activation(A_prev, parameters["W"+str(l)], parameters["b"+str(l)], activation = "sigmoid")
        caches.append(cache)
        
    AL, cache = linear_activation(A, parameters["W"+str(L)], parameters["b"+str(L)], activation = "relu")
    caches.append(cache)
    
    return AL, caches

In [6]:
def compute_cost(AL, Y):
    m  = Y.shape[1]
    cost = -np.sum(Y*np.log(AL)+(1-Y)*np.log(1-AL))
    cost = np.squeeze(cost)
    return cost

In [7]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = np.dot(dZ, A_prev.T)/m
    db = np.sum(dZ, axis = 1, keepdims = True)/m
    dA_prev = np.dot(W.T, dZ)
    
    return dA_prev, dW, db

In [8]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "sigmoid":
        dZ = sigmoid_backword(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    return dA_prev, dW, db

In [9]:
def l_model_backward(AL, Y ,caches):
    
    grads = {}
    L = len(caches)
    m = Y.shape[1]
    Y = Y.reshape(AL.shape)
    
    #Intialize
    dAL = -np.divide(Y, AL) + np.divide(1-Y, 1-AL)
    
    #Sigmoid Layer
    current_cache = caches[L-1]
    grads["dA"+str(L-1)], grads["dW"+str(L)], grads["db"+str(L)] = linear_backward(dAL, current_cache, activation = "sigmoid")
    
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA"+str(l+1)], current_cache, activation = "relu")
        grads["dA"+str(l)] = dA_prev_temp
        grads["dA"+str(l+1)] = dW_temp
        grads["dA"+str(l+1)] = db_temp
        
       
    return grads

In [10]:
def update_parameters(parameters, grads, learning_rate):
    
    L = len(parameters)/2
    
    for l in range(L):
        parameters["W"+str(l+1)] = parameters["W"+str(l+1)] - learning_rate*grads["dW"+str(l+1)]
        parameters["b"+str(l+1)] = parameters["b"+str(l+1)] - learning_rate*grads["db"+str(l+1)]
        
    return parameters
        